In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

Using TensorFlow backend.


In [67]:
# Good data
df = pd.read_csv('./data/kokil dec 6 reprepare/affcon_final_with_linguistics.csv')

In [89]:
# Messy data
df = pd.read_csv('./data/affcon_final_politeness_strategies_merge.csv')

In [41]:
df = df.dropna()

In [106]:
train, test = train_test_split(df, test_size=0.2)

y_train_deception = to_categorical(train['Input.deception_quadrant_cat'].tolist())
y_train_rapport_c = to_categorical(train['affcon_rapport'].tolist())
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()

y_test_deception = to_categorical(test['Input.deception_quadrant_cat'].tolist())
y_test_rapport_c = to_categorical(test['affcon_rapport'].tolist())
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()

X_train = train.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

X_test = test.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

### Individual classifiers are Random Forests

In [83]:
# Game move classifier
clf_gamemove = RandomForestClassifier(max_depth=2, random_state=0)
clf_gamemove.fit(X_train, y_train_gamemove)
y_pred_gamemove = clf_gamemove.predict(X_train)
y_pred_test_gamemove = clf_gamemove.predict(X_test)
print(classification_report(y_pred_gamemove, y_train_gamemove))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.84      0.91      3210

    accuracy                           0.84      3210
   macro avg       0.50      0.42      0.46      3210
weighted avg       1.00      0.84      0.91      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [107]:
# Rapport classifier
clf_rapport = RandomForestClassifier(max_depth=2, random_state=0)
clf_rapport.fit(X_train, y_train_rapport)
y_pred_rapport = clf_rapport.predict(X_train)
y_pred_test_rapport = clf_rapport.predict(X_test)
print(classification_report(y_pred_rapport, y_train_rapport))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.83      0.91      3210

    accuracy                           0.83      3210
   macro avg       0.50      0.42      0.45      3210
weighted avg       1.00      0.83      0.91      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
# Share Information classifier
clf_shareinfo = RandomForestClassifier(max_depth=2, random_state=0)
clf_shareinfo.fit(X_train, y_train_share_information)
y_pred_shareinfo = clf_shareinfo.predict(X_train)
y_pred_test_shareinfo = clf_shareinfo.predict(X_test)
print(classification_report(y_pred_shareinfo, y_train_share_information))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.81      0.89      3210

    accuracy                           0.81      3210
   macro avg       0.50      0.40      0.45      3210
weighted avg       1.00      0.81      0.89      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [86]:
# Reasoning classifier
clf_reasoning = RandomForestClassifier(max_depth=2, random_state=0)
clf_reasoning.fit(X_train, y_train_reasoning)
y_pred_reasoning = clf_reasoning.predict(X_train)
y_pred_test_reasoning = clf_reasoning.predict(X_test)
print(classification_report(y_pred_reasoning, y_train_reasoning))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.80      0.89      3210

    accuracy                           0.80      3210
   macro avg       0.50      0.40      0.44      3210
weighted avg       1.00      0.80      0.89      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
pred_df_arr = []
for i in range(0, len(y_pred_gamemove)):
    pred_obj = {}
    pred_obj['gamemove'] = y_pred_gamemove[i]
    pred_obj['reasoning'] = y_pred_reasoning[i]
    pred_obj['shareinfo'] = y_pred_shareinfo[i]
    pred_obj['rapport'] = y_pred_rapport[i]
    
    pred_df_arr.append(pred_obj)
    
pred_df = pd.DataFrame(pred_df_arr)

In [110]:
pred_test_df_arr = []
for i in range(0, len(y_pred_test_gamemove)):
    pred_obj = {}
    pred_obj['gamemove'] = y_pred_test_gamemove[i]
    pred_obj['reasoning'] = y_pred_test_reasoning[i]
    pred_obj['shareinfo'] = y_pred_test_shareinfo[i]
    pred_obj['rapport'] = y_pred_test_rapport[i]
    
    pred_test_df_arr.append(pred_obj)
    
pred_test_df = pd.DataFrame(pred_test_df_arr)

In [51]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [112]:
# Combined model

inputB = Input(shape=(pred_df.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
m = Model(inputs=inputB, outputs=c)

m.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m, precision_m,recall_m])
history = m.fit(x=pred_df, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df,y_test_deception), callbacks=[callback])

Epoch 1/32
51/51 [==============================] - 0s 8ms/step - loss: 7.7125 - acc: 0.0458 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.0523 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/32
51/51 [==============================] - 0s 4ms/step - loss: 7.7125 - acc: 0.0458 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.0523 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00


### Individual Classifiers are MLP

In [68]:
y_train_deception = to_categorical(train['Input.deception_quadrant_cat'].tolist())
y_train_rapport = to_categorical(train['affcon_rapport'].tolist())
y_train_share_information = to_categorical(train['affcon_shareinformation'].tolist())
y_train_reasoning = to_categorical(train['affcon_reasoning'].tolist())
y_train_gamemove = to_categorical(train['affcon_gamemove'].tolist())

y_test_deception = to_categorical(test['Input.deception_quadrant_cat'].tolist())
y_test_rapport = to_categorical(test['affcon_rapport'].tolist())
y_test_share_information = to_categorical(test['affcon_shareinformation'].tolist())
y_test_reasoning = to_categorical(test['affcon_reasoning'].tolist())
y_test_gamemove = to_categorical(test['affcon_gamemove'].tolist())

X_train = train.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

X_test = test.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

In [69]:
# Game move model
inputB = Input(shape=(X_train.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
gamemove_model = Model(inputs=inputB, outputs=c)

gamemove_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = gamemove_model.fit(x=X_train, y=y_train_gamemove, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = gamemove_model.evaluate(X_test, y_test_gamemove, verbose=0)
print(precision, recall, f1_score)

y_pred_gamemove = gamemove_model.predict(X_train)
y_pred_test_gamemove = gamemove_model.predict(X_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
# Reasoning model
inputB = Input(shape=(X_train.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
gamemove_model = Model(inputs=inputB, outputs=c)

gamemove_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = gamemove_model.fit(x=X_train, y=y_train_gamemove, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = gamemove_model.evaluate(X_test, y_test_gamemove, verbose=0)
print(precision, recall, f1_score)

y_pred_gamemove = gamemove_model.predict(X_train)
y_pred_test_gamemove = gamemove_model.predict(X_test)

### Individual classifiers are LSTMs

Merge classifiers by Add

In [90]:
train, test = train_test_split(df, test_size=0.2)

y_train_deception = train['Input.deception_quadrant_cat'].tolist()
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()

y_test_deception = test['Input.deception_quadrant_cat'].tolist()
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()

X_train_col = train['Input.full_text']

X_test_col = test['Input.full_text']

In [71]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [91]:
le = LabelEncoder()

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [92]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

tok.fit_on_texts(X_test_col)
X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

In [93]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)
def create_lstm():
    Inp = Input(name='inputs', shape=[max_len])
    x = Embedding(max_words, 50, input_length=max_len)(Inp)
    x = LSTM(64, name='LSTM_01')(x)
    x = Dropout(0.5, name='Dropout')(x)
    x = Dense(128, activation='relu',name='Dense_01')(x)
    # x = Dropout(0.5,name='Dropout')(x)
    out = Dense(1,activation='sigmoid', name='output')(x)
    model = Model(inputs=Inp, outputs=out)
    
    return model

In [94]:
# Rapport model
rapport_model = create_lstm()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
rapport_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_rapport), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 41ms/step - loss: 0.6676 - accuracy: 0.6028 - f1_m: 0.7494 - recall_m: 0.9906 - precision_m: 0.6056 - val_loss: 0.6620 - val_accuracy: 0.5994 - val_f1_m: 0.7493 - val_recall_m: 0.9995 - val_precision_m: 0.5998
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.6461 - accuracy: 0.6028 - f1_m: 0.7276 - recall_m: 0.8928 - precision_m: 0.6206 - val_loss: 0.6629 - val_accuracy: 0.5975 - val_f1_m: 0.7427 - val_recall_m: 0.9715 - val_precision_m: 0.6017


In [95]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

In [108]:
# Game move model
gamemove_model = create_lstm()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
gamemove_model.fit(X_train,y_train_gamemove,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_gamemove), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 40ms/step - loss: 0.6000 - accuracy: 0.7171 - f1_m: 0.8335 - recall_m: 0.9946 - precision_m: 0.7188 - val_loss: 0.5979 - val_accuracy: 0.7163 - val_f1_m: 0.8346 - val_recall_m: 1.0000 - val_precision_m: 0.7171
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.5828 - accuracy: 0.7199 - f1_m: 0.8365 - recall_m: 0.9997 - precision_m: 0.7197 - val_loss: 0.5953 - val_accuracy: 0.7160 - val_f1_m: 0.8343 - val_recall_m: 0.9995 - val_precision_m: 0.7170
Epoch 3/15
99/99 [==============================] - 3s 35ms/step - loss: 0.5737 - accuracy: 0.7202 - f1_m: 0.8359 - recall_m: 0.9968 - precision_m: 0.7209 - val_loss: 0.6053 - val_accuracy: 0.7138 - val_f1_m: 0.8324 - val_recall_m: 0.9933 - val_precision_m: 0.7172


In [97]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

In [98]:
# Reasoning model
reasoning_model = create_lstm()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
reasoning_model.fit(X_train,y_train_reasoning,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_reasoning), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 39ms/step - loss: 0.5571 - accuracy: 0.7609 - f1_m: 0.8627 - recall_m: 0.9957 - precision_m: 0.7626 - val_loss: 0.5446 - val_accuracy: 0.7697 - val_f1_m: 0.8693 - val_recall_m: 1.0000 - val_precision_m: 0.7694
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.5293 - accuracy: 0.7636 - f1_m: 0.8652 - recall_m: 0.9990 - precision_m: 0.7639 - val_loss: 0.5330 - val_accuracy: 0.7697 - val_f1_m: 0.8685 - val_recall_m: 0.9932 - val_precision_m: 0.7724
Epoch 3/15
99/99 [==============================] - 3s 35ms/step - loss: 0.5194 - accuracy: 0.7685 - f1_m: 0.8661 - recall_m: 0.9857 - precision_m: 0.7733 - val_loss: 0.5446 - val_accuracy: 0.7602 - val_f1_m: 0.8591 - val_recall_m: 0.9541 - val_precision_m: 0.7822


In [99]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

In [100]:
# Share Information model
shareinfo_model = create_lstm()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
shareinfo_model.fit(X_train,y_train_share_information,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_share_information), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 39ms/step - loss: 0.6808 - accuracy: 0.5557 - f1_m: 0.6170 - recall_m: 0.7302 - precision_m: 0.5704 - val_loss: 0.6844 - val_accuracy: 0.5559 - val_f1_m: 0.5412 - val_recall_m: 0.5005 - val_precision_m: 0.5936
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.6425 - accuracy: 0.6253 - f1_m: 0.5987 - recall_m: 0.5538 - precision_m: 0.6780 - val_loss: 0.7579 - val_accuracy: 0.5543 - val_f1_m: 0.5753 - val_recall_m: 0.5743 - val_precision_m: 0.5800


In [101]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

In [102]:
# Deception model
deception_model = create_lstm()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
deception_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_deception), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 40ms/step - loss: 0.2271 - accuracy: 0.9454 - f1_m: 0.9715 - recall_m: 0.9968 - precision_m: 0.9484 - val_loss: 0.1926 - val_accuracy: 0.9520 - val_f1_m: 0.9754 - val_recall_m: 1.0000 - val_precision_m: 0.9522
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.2057 - accuracy: 0.9483 - f1_m: 0.9735 - recall_m: 1.0000 - precision_m: 0.9485 - val_loss: 0.1913 - val_accuracy: 0.9520 - val_f1_m: 0.9754 - val_recall_m: 1.0000 - val_precision_m: 0.9522
Epoch 3/15
99/99 [==============================] - 3s 35ms/step - loss: 0.1973 - accuracy: 0.9483 - f1_m: 0.9734 - recall_m: 1.0000 - precision_m: 0.9483 - val_loss: 0.1930 - val_accuracy: 0.9520 - val_f1_m: 0.9754 - val_recall_m: 1.0000 - val_precision_m: 0.9522


In [103]:
# One hot encoding
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj = {}
    pred_obj['gamemove'] = gamemove_pred[i][0]
    pred_obj['reasoning'] = reasoning_pred[i][0]
    pred_obj['shareinfo'] = shareinfo_pred[i][0]
    pred_obj['rapport'] = rapport_pred[i][0]
    
    pred_df_arr.append(pred_obj)
    
pred_df = pd.DataFrame(pred_df_arr)

In [104]:
pred_test_df_arr = []
for i in range(0, len(gamemove_pred_test)):
    pred_obj = {}
    pred_obj['gamemove'] = gamemove_pred_test[i][0]
    pred_obj['reasoning'] = reasoning_pred_test[i][0]
    pred_obj['shareinfo'] = shareinfo_pred_test[i][0]
    pred_obj['rapport'] = rapport_pred_test[i][0]
    
    pred_test_df_arr.append(pred_obj)
    
pred_test_df = pd.DataFrame(pred_test_df_arr)

In [112]:
pred_df.head()

,gamemove,reasoning,shareinfo,rapport
0,0.709489,0.840170,0.837677,0.639576
1,0.683541,0.853558,0.680558,0.623211
2,0.577929,0.632894,0.490597,0.564213
3,0.711058,0.816315,0.778096,0.708829
4,0.618718,0.720346,0.556693,0.500282


In [113]:
from sklearn.utils import class_weight
class_weight = {'gamemove': 50,
                'reasoning': 50.,
                'shareinfo': 10, 'rapport': 2}

In [121]:
inputB = Input(shape=(pred_df.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='softmax')(c)
full_model_weight = Model(inputs=inputB, outputs=c)

full_model_weight.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])

history = full_model_weight.fit(x=pred_df, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])

Epoch 1/32
158/158 [==============================] - 1s 5ms/step - loss: 0.6931 - acc: 0.2623 - f1_m: 0.9698 - precision_m: 0.9499 - recall_m: 0.9947 - val_loss: 0.6931 - val_acc: 0.9412 - val_f1_m: 0.9688 - val_precision_m: 0.9400 - val_recall_m: 1.0000
Epoch 2/32
158/158 [==============================] - 1s 4ms/step - loss: 0.6931 - acc: 0.4348 - f1_m: 0.9742 - precision_m: 0.9501 - recall_m: 1.0000 - val_loss: 0.6931 - val_acc: 0.9412 - val_f1_m: 0.9688 - val_precision_m: 0.9400 - val_recall_m: 1.0000
Epoch 3/32
158/158 [==============================] - 1s 4ms/step - loss: 0.6931 - acc: 0.4581 - f1_m: 0.9742 - precision_m: 0.9501 - recall_m: 1.0000 - val_loss: 0.6931 - val_acc: 0.0608 - val_f1_m: 0.9688 - val_precision_m: 0.9400 - val_recall_m: 1.0000


In [118]:
# Joint Model with one hot encoding 
inputB = Input(shape=(pred_df.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='softmax')(c)
full_model = Model(inputs=inputB, outputs=c)

full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = full_model.fit(x=pred_df, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])

Epoch 1/32
158/158 [==============================] - 1s 5ms/step - loss: 0.9902 - acc: 0.0499 - f1_m: 0.9740 - precision_m: 0.9498 - recall_m: 1.0000 - val_loss: 0.7750 - val_acc: 0.0588 - val_f1_m: 0.9688 - val_precision_m: 0.9400 - val_recall_m: 1.0000
Epoch 2/32
158/158 [==============================] - 1s 3ms/step - loss: 0.7255 - acc: 0.0623 - f1_m: 0.9743 - precision_m: 0.9503 - recall_m: 1.0000 - val_loss: 0.7021 - val_acc: 0.1195 - val_f1_m: 0.9688 - val_precision_m: 0.9400 - val_recall_m: 1.0000
Epoch 3/32
158/158 [==============================] - 1s 3ms/step - loss: 0.6974 - acc: 0.2384 - f1_m: 0.9741 - precision_m: 0.9499 - recall_m: 1.0000 - val_loss: 0.6954 - val_acc: 0.3642 - val_f1_m: 0.9688 - val_precision_m: 0.9400 - val_recall_m: 1.0000
Epoch 4/32
158/158 [==============================] - 1s 3ms/step - loss: 0.6950 - acc: 0.4377 - f1_m: 0.9742 - precision_m: 0.9501 - recall_m: 1.0000 - val_loss: 0.6949 - val_acc: 0.4877 - val_f1_m: 0.9688 - val_precision_m: 0.9400

In [109]:
# Joint Model with model Add
commonInput = Input(shape=[max_len])

out1 = gamemove_model(commonInput)    
out2 = reasoning_model(commonInput)   
out3 = shareinfo_model(commonInput)
out4 = rapport_model(commonInput)
mergedOut = Add()([out1,out2,out3,out4])

mergedOut = Flatten()(mergedOut)    
mergedOut = Dense(256, activation='relu')(mergedOut)
mergedOut = Dropout(.5)(mergedOut)
mergedOut = Dense(128, activation='relu')(mergedOut)
mergedOut = Dropout(.35)(mergedOut)
mergedOut = Dense(2, activation='softmax')(mergedOut)  #Cuz binary

mergedModel = Model(commonInput, mergedOut)
mergedModel.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', f1_m, recall_m, precision_m])
shareinfo_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_split=0.2, callbacks=[early_stop])

Epoch 1/15
79/79 [==============================] - 3s 35ms/step - loss: 0.2425 - accuracy: 0.9423 - f1_m: 0.9692 - recall_m: 0.9916 - precision_m: 0.9502 - val_loss: 0.2294 - val_accuracy: 0.9412 - val_f1_m: 0.9693 - val_recall_m: 1.0000 - val_precision_m: 0.9407
Epoch 2/15
79/79 [==============================] - 3s 35ms/step - loss: 0.2008 - accuracy: 0.9501 - f1_m: 0.9744 - recall_m: 1.0000 - precision_m: 0.9503 - val_loss: 0.2459 - val_accuracy: 0.9412 - val_f1_m: 0.9693 - val_recall_m: 1.0000 - val_precision_m: 0.9407
